# Title matching

Given the actual titles of books on bookshelves (manually entered), queries each of the book information sources for the titles that they return; this is used in order to actually determine whether a true match is made or not when some book information is obtained

Saves the information to the correct place.

## Imports

In [ ]:
import os

import shelfy
import shelfy.models.scraper

## Load a bookshelf

In [ ]:
# Set directories
bookshelf_directory = shelfy.SHELFY_BASE_PATH + '/data/shelves/'
bookshelf_name = 'insight_0'
bookshelf_path = bookshelf_directory + bookshelf_name + '/titles'


# Read book titles in from plain text file
book_titles = []
with open(bookshelf_path, 'r') as file_handle:
    for book_title in file_handle:
        book_titles.append(book_title.replace('\n', ''))
        
print(book_titles)

## Perform the queries

In [ ]:
query_google_urls = [shelfy.models.scraper.get_google_search_url_from_query(book_title) for book_title in book_titles]

print(query_google_urls)

##### ISBN's

In [ ]:
isbns = []
for i in range(len(query_google_urls)):
    isbn = None
    while isbn == None:
        isbn = shelfy.models.scraper.get_isbn10_from_google_search(query_google_urls[i])
    print(isbn)
    isbns.append(isbn)
        

##### Google books